In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
reload(fd)
reload(sd)
reload(di)
reload(resultMd)

print('done')

In [ ]:
# Results
#max_depth default is 1 is too low, but a too high value will occur overfitting
max_depthChoice=4
# learning too low makes the process very slow
learning_rateChoice=1.2
n_estimatorsChoice=110

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Test with default hyperparameters

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = AdaBoostClassifier(random_state=42)
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('07-sklearn.ensemble.b-boosting','AdaBoostClassifier','1-Default', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('07-sklearn.ensemble.b-boosting','AdaBoostClassifier','1-Default', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)

# default result
# f1 train 0.0220
# f1 test : 0.0360

# Change estimator because default is too simple

In [ ]:
#%%script false

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, balanced_accuracy_score, f1_score
import numpy as np

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)
predictors = fd.getPredictors(dfTrx)

train_f1s=[]
test_f1s=[]
range= []
# too low max_depth -> no result
for max_depth in np.arange(1,7,1):
    print('max_depth',max_depth)

    estimator = DecisionTreeClassifier(max_depth=max_depth)
    modelClf = AdaBoostClassifier(random_state=42,estimator=estimator, n_estimators=n_estimatorsChoice, learning_rate=learning_rateChoice)
    modelClf.fit(x_train, y_train)
    train_preds = modelClf.predict(x_train)
    train_f1 = f1_score(y_train,train_preds)

    test_preds = modelClf.predict(x_test)
    test_f1 = f1_score(y_test,test_preds)
    #fd.show_confusion_matrix(y_train, train_preds)

    print("train_f1", train_f1)
    print("test_f1 ", test_f1)
    print("dif     ", train_f1-test_f1)
    train_f1s.append(train_f1)
    test_f1s.append(test_f1)
    range.append(max_depth)
    print('-----------------------')

fd.plt_train_test(range, train_f1s, "train", test_f1s,"test")

# choose estimator = DecisionTreeClassifier(max_depth=3) above overfitting

# Scaling choice
With adaboost the scaling has no influence ?

In [ ]:
%%script false
# scaling choice

import matplotlib.pyplot as plt
import numpy as np

predictors = fd.getPredictors(dfTrx)
parameters={}
#parameters['max_iter']=1000

modelClf = AdaBoostClassifier(random_state=42)
duration_in_s,f1Train,f1Test, scaler = fd.processModel(modelClf,dfTrx, predictors,[], parameters, scaler=None)
scalers = fd.getScalers()
for key in scalers:
    print(key)
    scaler=scalers.get(key)
    modelClf = AdaBoostClassifier(random_state=42)
    duration_in_s,f1Train,f1Test, scaler = fd.processModel(modelClf,dfTrx, predictors, [],parameters,scaler=scaler)

# scaler has no adding value for the result

# Hyperparameters tuning
starting point
n_estimators=50,learning_rate=1

In [ ]:
%%script false

from sklearn.model_selection import KFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, balanced_accuracy_score, f1_score
import numpy as np


x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)
predictors = fd.getPredictors(dfTrx)
train_f1s=[]
test_f1s=[]
range= []

for learning_rate in np.arange(0.5,2.0,0.1):
    print('learning_rate',learning_rate)
    # Instantiate model and k-fold
    estimator = DecisionTreeClassifier(max_depth=max_depthChoice)
    modelClf = AdaBoostClassifier(random_state=42,estimator=estimator, n_estimators=n_estimatorsChoice, learning_rate=learning_rate)
    modelClf.fit(x_train, y_train)
    train_preds = modelClf.predict(x_train)
    train_f1 = f1_score(y_train,train_preds)

    test_preds = modelClf.predict(x_test)
    test_f1 = f1_score(y_test,test_preds)
    #fd.show_confusion_matrix(y_train, train_preds)

    print("train_f1", train_f1)
    print("test_f1 ", test_f1)
    print("dif     ", train_f1-test_f1)
    train_f1s.append(train_f1)
    test_f1s.append(test_f1)
    range.append(learning_rate)
    print('-----------------------')

fd.plt_train_test(range, train_f1s, "train", test_f1s,"test")

In [ ]:
%%script false

from sklearn.model_selection import KFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, balanced_accuracy_score, f1_score
import numpy as np


x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)
predictors = fd.getPredictors(dfTrx)
train_f1s=[]
test_f1s=[]
range= []

for n_estimators in np.arange(30,161,10):
    print('n_estimators',n_estimators)
    # Instantiate model and k-fold
    estimator = DecisionTreeClassifier(max_depth=max_depthChoice)
    modelClf = AdaBoostClassifier(random_state=42,estimator=estimator, n_estimators=n_estimators, learning_rate=learning_rateChoice)
    modelClf.fit(x_train, y_train)
    train_preds = modelClf.predict(x_train)
    train_f1 = f1_score(y_train,train_preds)

    test_preds = modelClf.predict(x_test)
    test_f1 = f1_score(y_test,test_preds)
    #fd.show_confusion_matrix(y_train, train_preds)

    print("train_f1", train_f1)
    print("test_f1 ", test_f1)
    print("dif     ", train_f1-test_f1)
    train_f1s.append(train_f1)
    test_f1s.append(test_f1)
    range.append(n_estimators)
    print('-----------------------')

fd.plt_train_test(range, train_f1s, "train", test_f1s,"test")

In [ ]:
%%script false
from scipy.stats import randint

modelClf = AdaBoostClassifier(random_state=42)
# starting point n_estimators=100,  max_depth=7 
dic_param={
    'n_estimators': randint(300,750),
    'learning_rate':[0.1]
}
res=fd.hyperparameterSelectionRandomizedSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)
#{'learning_rate': 2, 'n_estimators': 514}
#0.1096848223301722

In [ ]:
%%script false
modelClf = AdaBoostClassifier(random_state=42)
 
dic_param={
    'n_estimators':[430,450,480,500,530,550,600],
    'learning_rate':[0.1]
}
res=fd.hyperparameterSelectionGridSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)

#{'learning_rate': 2, 'n_estimators': 450}
#0.1096848223301722
#scoref1 0.10342393222732087


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

estimator = DecisionTreeClassifier(max_depth=max_depthChoice)
modelClf = AdaBoostClassifier(random_state=42)
parameters={'learning_rate': learning_rateChoice, 'n_estimators':n_estimatorsChoice, 'estimator':estimator}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('07-sklearn.ensemble.b-boosting','AdaBoostClassifier','2-After tuning', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('07-sklearn.ensemble.b-boosting','AdaBoostClassifier','2-After tuning', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test,'../imgs/FD07C-AdaBoostClassifierProbaHistogram' if saveImage else None)



In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result('07-sklearn.ensemble.b-boosting','AdaBoostClassifier','2-After tuning', f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])

